In [1]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  

GPT_MODEL = "gpt-4o"
client = OpenAI()

In [3]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
# 재시도 데코레이터: 함수 실행 실패 시 지수 백오프로 최대 3번 재시도
# 대기 시간은 1초부터 시작하여 최대 40초까지 랜덤하게 증가

def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    # ChatGPT API에 요청을 보내는 함수
    # messages: 대화 내용
    # tools: 사용할 도구 (선택적)
    # tool_choice: 선택할 도구 (선택적)
    # model: 사용할 GPT 모델 (기본값: GPT_MODEL)

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        # OpenAI API를 통해 채팅 완성 요청
        # 지정된 모델, 메시지, 도구, 도구 선택을 사용하여 응답 생성
        return response
        # 생성된 응답 반환

    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        # 요청 실패 시 오류 메시지 출력
        return e
        # 발생한 예외 반환

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

In [5]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message.content


'Sure, could you please provide me with the city and state (or country) you are interested in?'

In [6]:
messages.append({"role": "user", "content": "I'm in Glasgow, Scotland."})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
print(assistant_message.content)
if assistant_message.content != None:
    messages.append(assistant_message)



None


In [7]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Glasgow, Scotland over the next x days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
print(assistant_message.content)


Could you please specify the number of days for which you would like the weather forecast in Glasgow, Scotland?


In [8]:
messages.append({"role": "user", "content": "5 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
print(chat_response.choices[0].message.content)


None


In [9]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
)
print(chat_response.choices[0].message.content)

None


In [10]:
# if we don't force the model to use get_n_day_weather_forecast it may not
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools
)
print(chat_response.choices[0].message.content)

None


In [11]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in San Francisco and Glasgow over the next 4 days"})
chat_response = chat_completion_request(
    messages, tools=tools, model=GPT_MODEL
)

assistant_message = chat_response.choices[0].message.tool_calls
print(assistant_message)


[ChatCompletionMessageToolCall(id='call_uU2pjgFOw75dWgH0PCz0MeU9', function=Function(arguments='{"location": "San Francisco, CA", "format": "fahrenheit", "num_days": 4}', name='get_n_day_weather_forecast'), type='function'), ChatCompletionMessageToolCall(id='call_QzBvBcx1vHDftUlNKlfJ0PEQ', function=Function(arguments='{"location": "Glasgow, UK", "format": "celsius", "num_days": 4}', name='get_n_day_weather_forecast'), type='function')]


In [12]:
print(chat_response.choices[0].message.content)

None


In [13]:
import sqlite3

conn = sqlite3.connect("data/Chinook.db")
print("Opened database successfully")

Opened database successfully


In [14]:
def get_table_names(conn):
    """테이블 이름 목록을 반환합니다."""
    table_names = []
    # SQLite 시스템 테이블에서 모든 테이블 이름을 조회
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """주어진 테이블의 열 이름 목록을 반환합니다."""
    column_names = []
    # PRAGMA 명령을 사용하여 테이블의 열 정보를 조회
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])  # col[1]은 열 이름을 나타냄
    return column_names


def get_database_info(conn):
    """데이터베이스의 각 테이블에 대한 테이블 이름과 열 이름을 포함하는 딕셔너리 리스트를 반환합니다."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        # 각 테이블에 대해 테이블 이름과 열 이름을 딕셔너리로 저장
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

In [15]:
# 데이터베이스 연결(conn)을 사용하여 데이터베이스 스키마 정보를 가져옵니다
database_schema_dict = get_database_info(conn)

# 데이터베이스 스키마 정보를 문자열로 변환합니다
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

# 위 코드는 다음과 같은 작업을 수행합니다:
# 1. get_database_info 함수를 호출하여 데이터베이스의 모든 테이블과 열 정보를 가져옵니다.
# 2. 가져온 정보를 순회하면서 각 테이블에 대해 다음 형식의 문자열을 생성합니다:
#    "Table: [테이블 이름]
#     Columns: [열1 이름], [열2 이름], ..."
# 3. 생성된 모든 문자열을 줄바꿈(\n)으로 연결하여 하나의 큰 문자열로 만듭니다.
# 이렇게 생성된 문자열은 데이터베이스의 전체 구조를 간단히 표현합니다.

In [17]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "ask_database",
            "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

In [16]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

In [20]:
# Step #1: Prompt with content that may result in function call. In this case the model can identify the information requested by the user is potentially available in the database schema passed to the model in Tools description. 
messages = [{
    "role":"user", 
    "content": "직원은 몇명이지?"
}]

response = client.chat.completions.create(
    model='gpt-4o', 
    messages=messages, 
    tools= tools, 
    tool_choice="auto"
)

# Append the message to messages list
response_message = response.choices[0].message 
messages.append(response_message)

print(response_message.content)

None


In [19]:
# Step 2: determine if the response from the model includes a tool call.   
tool_calls = response_message.tool_calls
if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)  
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    tool_query_string = eval(tool_calls[0].function.arguments)['query']
    
    # Step 3: Call the function and retrieve results. Append the results to the messages list.      
    if tool_function_name == 'ask_database':
        results = ask_database(conn, tool_query_string)
        
        messages.append({
            "role":"tool", 
            "tool_call_id":tool_call_id, 
            "name": tool_function_name, 
            "content":results
        })
        
        # Step 4: Invoke the chat completions API with the function response appended to the messages list
        # Note that messages with role 'tool' must be a response to a preceding message with 'tool_calls'
        model_response_with_function_call = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print(model_response_with_function_call.choices[0].message.content)
    else: 
        print(f"Error: function {tool_function_name} does not exist")
else: 
    # Model did not identify a function to call, result can be returned to the user 
    print(response_message.content) 

현재 직원 수는 총 8명입니다.
